In [17]:
%load_ext autoreload
%autoreload 2

import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *

from annotation_utilities import *
from registration_utilities import *

import matplotlib.pyplot as plt
%matplotlib inline

import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from conversion import *

In [19]:
stack = 'MD592'

In [20]:
downsample_factor = 32
xy_pixel_distance_downsampled = xy_pixel_distance_lossless * downsample_factor
voxel_z_size = section_thickness / xy_pixel_distance_downsampled

In [21]:
sections_to_filenames = metadata_cache['sections_to_filenames'][stack]
filenames_to_sections = {f: s for s, f in sections_to_filenames.iteritems() 
                        if f not in ['Placeholder', 'Nonexisting', 'Rescan']}

In [22]:
import cPickle as pickle
Ts = pickle.load(open('/home/yuncong/CSHL_data_processed/%(stack)s/%(stack)s_elastix_output/%(stack)s_transformsTo_anchor.pkl' % dict(stack=stack), 'r'))

Ts_inv_downsampled = {}
for fn, T0 in Ts.iteritems():
    T = T0.copy()
    T[:2, 2] = T[:2, 2] * 32 / downsample_factor
    Tinv = np.linalg.inv(T)
    Ts_inv_downsampled[fn] = Tinv

In [23]:
# Compute vertices of outer shell

outer_contours = []

for fn, sec in filenames_to_sections.iteritems():
    if fn in ['Placeholder', 'Nonexisting', 'Rescan']:
        continue
        
    mask = imread(os.path.join(data_dir, stack, stack+'_mask_unsorted', fn + '_mask.png'))
    contours = find_contour_points(mask, sample_every=5)[255]
    
    if len(contours) > 1:
        outer_contour = sorted(contours, key=lambda x: len(x))[-1] # largest contour
    else:
        outer_contour = contours[0]
        
#     plt.plot(outer_contour[:,0], outer_contour[:,1]);
        
    z = voxel_z_size * (sec - 1)
    n = len(outer_contour)
    
    vertices_on_aligned = np.dot(Ts_inv_downsampled[fn], np.c_[outer_contour*32/downsample_factor, np.ones((n,))].T).T[:, :2]
    vertices_on_aligned_3d = np.c_[vertices_on_aligned, z*np.ones((n,))]
    outer_contours.append(vertices_on_aligned_3d)

In [24]:
outerContour_volume, (oc_xmin,oc_xmax,oc_ymin,oc_ymax,oc_zmin,oc_zmax) = \
contours_to_volume(contours_grouped_by_label={1: outer_contours}, 
                   interpolation_direction='z', return_shell=True, len_interval=5)

In [ ]:
zs = range(0, outerContour_volume.shape[2], 3)

fig, axes = plt.subplots(len(zs)/3+1, 3, figsize=(20,100) , squeeze=True);
axes = axes.flatten()
for i, z in enumerate(zs):
#     print len(find_contour_points(outerContour_volume[..., z])[1])
    axes[i].imshow(outerContour_volume[..., z], vmin=0, vmax=1, cmap=plt.cm.gray);
    axes[i].set_title('z = %d' % z);
plt.tight_layout();
plt.show();

In [25]:
# Output

bp.pack_ndarray_file(outerContour_volume, os.path.join(VOLUME_ROOTDIR, '%(stack)s/%(stack)s_down%(down)d_outerContourVolume.bp' % \
                                          dict(down=downsample_factor, stack=stack)))

with open(os.path.join(VOLUME_ROOTDIR, '%(stack)s/%(stack)s_down%(down)d_outerContourVolume_bbox.txt' % \
                      dict(down=downsample_factor, stack=stack)), 'w') as f:
        f.write((' '.join(['%d']*6)+'\n') % (oc_xmin, oc_xmax, oc_ymin, oc_ymax, oc_zmin, oc_zmax))